<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/GEMINI_FP_AGENT_JUNE2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import time
from datetime import datetime
import json
import os

# --- Google Colab / Gemini API Imports ---
try:
    from google.colab import userdata
    GOOGLE_API_KEY = userdata.get('GEMINI')
    import google.generativeai as genai
    genai.configure(api_key=GOOGLE_API_KEY)
    print("Google Generative AI configured successfully using Colab Secrets.")
except ImportError:
    print("Not running in Google Colab. Attempting to get 'GEMINI' environment variable.")
    GOOGLE_API_KEY = os.getenv('GEMINI')
    if not GOOGLE_API_KEY:
        raise ValueError("GEMINI API Key not found. Please set it as an environment variable or Colab Secret.")
    import google.generativeai as genai
    genai.configure(api_key=GOOGLE_API_KEY)
    print("Google Generative AI configured successfully using environment variable.")

# --- Configuration ---
class AgentConfig:
    LLM_PROVIDER: str = "google"
    LLM_MODEL_NAME: str = "gemini-1.5-flash"


class AgentMemory:
    """
    Represents the Memory keystone of our Agentic AI.
    Stores various types of information relevant to flight planning.
    """
    def __init__(self):
        self.long_term_data = {
            "airport_database": {
                "YUL": {"name": "Montreal-Trudeau Intl.", "coordinates": "45.4706° N, 73.7408° W", "timezone": "EDT"},
                "JFK": {"name": "John F. Kennedy Intl.", "coordinates": "40.6413° N, 73.7781° W", "timezone": "EDT"},
                "LAX": {"name": "Los Angeles Intl.", "coordinates": "33.9416° N, 118.4085° W", "timezone": "PDT"},
                "LHR": {"name": "London Heathrow Airport", "coordinates": "51.4700° N, 0.4543° W", "timezone": "BST"},
                "CDG": {"name": "Paris Charles de Gaulle Airport", "coordinates": "49.0097° N, 2.5479° E", "timezone": "CEST"},
                "BKK": {"name": "Bangkok Suvarnabhumi Airport", "coordinates": "13.6899° N, 100.7501° E", "timezone": "ICT"}
            },
            "aircraft_specs": {
                "Boeing 737": {"range_km": 5000, "speed_kph": 850, "fuel_burn_rate_L_per_km": 5, "capacity": 180},
                "Airbus A320": {"range_km": 6100, "speed_kph": 830, "fuel_burn_rate_L_per_km": 4.5, "capacity": 150},
                "Boeing 787": {"range_km": 14000, "speed_kph": 900, "fuel_burn_rate_L_per_km": 8, "capacity": 300},
                "Boeing 767": {"range_km": 11000, "speed_kph": 880, "fuel_burn_rate_L_per_km": 7, "capacity": 250},
                "Boeing 777": {"range_km": 16000, "speed_kph": 920, "fuel_burn_rate_L_per_km": 9, "capacity": 400}
            },
            "past_successful_flights": [],
            "regulations": "Standard ICAO flight rules, regional airspace restrictions, ETOPS considerations for twin-engine aircraft on long overwater flights. Prioritize safety and fuel efficiency."
        }
        self.short_term_data = {}

    def store(self, key, value, is_long_term=False):
        if is_long_term:
            self.long_term_data[key] = value
        else:
            self.short_term_data[key] = value

    def retrieve(self, key, is_long_term=False):
        if is_long_term and key in self.long_term_data:
            return self.long_term_data[key]
        elif not is_long_term and key in self.short_term_data:
            return self.short_term_data[key]
        return None

class LLMReasoningModule:
    """
    Integrates a Large Language Model (specifically AgentConfig.LLM_MODEL_NAME)
    for complex reasoning and planning.
    """
    def __init__(self, memory):
        self.memory = memory
        self.llm_model = genai.GenerativeModel(AgentConfig.LLM_MODEL_NAME)
        print(f"  [LLM Integration]: Initialized LLM client using model: {AgentConfig.LLM_MODEL_NAME}.")

    def call_llm(self, prompt_text):
        """
        Makes an actual API call to the configured Gemini LLM.
        """
        print("\n--- Invoking Actual LLM API Call ---")
        print(f"  Prompt sent to LLM:\n---START PROMPT---\n{prompt_text}\n---END PROMPT---")

        try:
            response = self.llm_model.generate_content(prompt_text)
            llm_raw_response_text = response.text
            print("  LLM Raw Response Text:", llm_raw_response_text)

            # Preprocess LLM response to strip markdown code block
            if llm_raw_response_text.strip().startswith("```json"):
                llm_json_string = llm_raw_response_text.strip()[len("```json"):].strip()
                if llm_json_string.endswith("```"):
                    llm_json_string = llm_json_string[:-len("```")].strip()
            else:
                llm_json_string = llm_raw_response_text.strip()

            try:
                llm_parsed_response = json.loads(llm_json_string)
                print("  LLM Parsed JSON Response:", json.dumps(llm_parsed_response, indent=2))
                return llm_parsed_response
            except json.JSONDecodeError as e:
                print(f"  [LLM Error]: Failed to parse LLM response as JSON: {e}")
                print(f"  Raw string attempted to parse: '{llm_json_string}'")
                return {"thought_process": "LLM response was not valid JSON after stripping markdown.", "flight_plan": None, "status": "parse_error"}

        except Exception as e:
            print(f"  [LLM Error]: An error occurred during LLM API call: {e}")
            return {"thought_process": f"LLM API call failed: {e}", "flight_plan": None, "status": "api_error"}


    def analyze_and_plan(self, goal, flight_details, current_weather):
        """
        Constructs the prompt and calls the actual LLM API.
        Parses the LLM's response to get the structured plan.
        """
        origin_code = flight_details.get("origin")
        destination_code = flight_details.get("destination")
        aircraft_type = flight_details.get("aircraft_type")
        pax_count = flight_details.get("pax_count")
        cargo_weight_kg = flight_details.get("cargo_weight_kg", 0)
        priority = flight_details.get('priority', 'None')

        airport_db = self.memory.retrieve("airport_database", is_long_term=True)
        aircraft_specs = self.memory.retrieve("aircraft_specs", is_long_term=True)
        regulations = self.memory.retrieve("regulations", is_long_term=True)

        origin_info = airport_db.get(origin_code, {})
        destination_info = airport_db.get(destination_code, {})
        aircraft_info = aircraft_specs.get(aircraft_type, {})

        # Construct a detailed prompt for the LLM, explicitly requesting JSON
        # The prompt will refer to the agent by its new, clear name
        prompt = f"""
        You are an advanced AI Flight Planning Assistant (named Aeronav AI Planner).
        Your task is to generate a comprehensive flight plan based on the user's request, current conditions, and your knowledge base.
        The output MUST be a JSON object containing a "thought_process" string, and a "flight_plan" object (or null if planning fails), and a "status" field.
        Ensure all JSON keys are properly quoted with double quotes.

        User Goal: "{goal}"
        Flight Details:
        - Origin Airport Code: {origin_code} (Full Name: {origin_info.get('name', 'N/A')}, Coordinates: {origin_info.get('coordinates', 'N/A')}, Timezone: {origin_info.get('timezone', 'N/A')})
        - Destination Airport Code: {destination_code} (Full Name: {destination_info.get('name', 'N/A')}, Coordinates: {destination_info.get('coordinates', 'N/A')}, Timezone: {destination_info.get('timezone', 'N/A')})
        - Aircraft Type: {aircraft_type}
        - Passengers: {pax_count if pax_count else 'N/A'}
        - Cargo Weight: {cargo_weight_kg} kg
        - Additional Request Priority: {priority}

        Current Environment Information:
        - Current UTC Time: {datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S UTC')}
        - Current Weather at Origin (approx): {current_weather}
        - Aircraft Specifications (for {aircraft_type}): {aircraft_info}
        - General Aviation Regulations Context: {regulations}

        Consider all factors like aircraft range, fuel requirements (including minimum 10% reserves), optimal altitude for efficiency/speed,
        weather impact, time zones, and relevant regulations.
        Provide a detailed route, estimated time (eta_hours), estimated fuel (estimated_fuel_L), optimal cruising altitude (altitude_ft), key waypoints, and any important notes for the flight crew.

        If a plan cannot be generated (e.g., impossible destination, insufficient aircraft range), set "flight_plan" to null and provide a detailed reason in "thought_process".

        Example of a SUCCESSFUL JSON output:
        {{
            "thought_process": "Analyzed request, checked range, considered weather. Determined direct route is feasible...",
            "flight_plan": {{
                "route": "Detailed description of the flight path.",
                "eta_hours": 6.8,
                "estimated_fuel_L": 35000,
                "altitude_ft": "39,000",
                "waypoints": ["YUL Departure", "Mid-Continent Crossing", "LAX Arrival"],
                "notes": "Monitor winds aloft. Suggest 1.5 hours fuel reserve."
            }},
            "status": "success"
        }}

        Example of a FAILED JSON output:
        {{
            "thought_process": "Aircraft range is insufficient for the requested distance.",
            "flight_plan": null,
            "status": "fail"
        }}

        Now, generate the JSON output for the given flight planning request:
        """

        # Call the actual LLM API
        llm_response = self.call_llm(prompt)

        if llm_response and llm_response.get("status") == "success" and llm_response.get("flight_plan") is not None:
            print(f"  [LLM Reasoning - Thought Process]:\n{llm_response['thought_process']}")
            return llm_response["flight_plan"]
        elif llm_response and llm_response.get("status") == "fail":
            print(f"  [LLM Reasoning]: LLM reported planning failure. Reason: {llm_response.get('thought_process', 'No specific reason provided by LLM.')}")
            return None
        else:
            print(f"  [LLM Reasoning]: LLM returned an invalid or unexpected response format. Reason: {llm_response.get('thought_process', 'Unknown error or malformed response.')}")
            return None

class AgentAction:
    """
    Represents the Action keystone.
    Executes plans and interacts with the environment.
    """
    def __init__(self, memory):
        self.memory = memory

    def execute_plan(self, plan):
        print(f"  [Action]: Initiating execution of flight plan: {plan['route']} (Estimated time: {plan['eta_hours']} hours)")
        time.sleep(1)

        actual_time_modifier = 1.0
        simulated_event = "smooth flight"
        # Using current time to vary simulation slightly
        if datetime.now().minute % 3 == 0:
            simulated_event = "minor headwind en route"
            actual_time_modifier = 1.05
        elif datetime.now().minute % 5 == 0:
            simulated_event = "air traffic control delay on departure"
            actual_time_modifier = 1.10
        elif datetime.now().minute % 7 == 0:
            simulated_event = "slight tailwind"
            actual_time_modifier = 0.95

        actual_time_hours = float(plan['eta_hours']) * actual_time_modifier
        outcome = {"status": "success", "actual_time_hours": round(actual_time_hours, 1), "event": simulated_event}
        print(f"  [Action]: Flight execution complete after a {simulated_event}. Outcome: {outcome['status']} in {outcome['actual_time_hours']} hours.")
        return outcome

    def communicate_status(self, message):
        print(f"  [Action]: Sending communication: '{message}'")

class FlightPlanningAgent:
    """
    Combines the Keystones (with LLM for Reasoning) to demonstrate the human action cycle.
    This is our Aeronav AI Planner agent, powered by the {LLM_MODEL_NAME}.
    """
    def __init__(self, name="Aeronav AI Planner (Powered by {model_name})".format(model_name=AgentConfig.LLM_MODEL_NAME)):
        self.name = name
        self.memory = AgentMemory()
        self.llm_reasoning = LLMReasoningModule(self.memory)
        self.action = AgentAction(self.memory)
        # Changed agent initialization message
        print(f"\n----- {self.name} Agent Initialized -----")
        print(f"  Our flight planning agent, leveraging a **Large Language Model ({AgentConfig.LLM_MODEL_NAME})** for dynamic reasoning and complex decision-making.")
        print("  Embodying Newton's physics, Galileo's data observation, Einstein's system dynamics,")
        print("  and Hinton's deep learning for robust flight planning.")
        print("---------------------------------------")

    def run_flight_planning_cycle(self, goal, flight_details):
        print(f"\n[{datetime.now().strftime('%Y-%m-%d %H:%M:%S EDT')}] Agent's Goal: \"{goal}\"")

        # 1. Input (Gather Information)
        print("\n--- Step 1: INPUT (Gathering Information) ---")
        self.memory.store("current_goal", goal)
        self.memory.store("flight_details", flight_details)
        print(f"  [Input]: Received flight request: {flight_details}")

        # Simulate getting real-time weather.
        current_weather = {"temperature_c": 15, "wind_kph": 30, "visibility_km": 8, "weather": "partly cloudy with moderate winds"}
        self.memory.store("current_weather", current_weather)
        print(f"  [Input]: Fetched current weather: {current_weather}")

        # 2. Think & Plan (LLM analyzes options, chooses approach, and outlines steps)
        print("\n--- Step 2 & 3: THINK & PLAN (LLM Analyzing and Outlining) ---")
        current_weather_for_llm = self.memory.retrieve("current_weather")
        final_plan = self.llm_reasoning.analyze_and_plan(goal, flight_details, current_weather_for_llm)

        if final_plan:
            self.memory.store("final_flight_plan", final_plan)
            print(f"  [Agent]: LLM has generated a final flight plan: {final_plan}")
            self.action.communicate_status("LLM-generated flight plan ready for execution.")
        else:
            print("  [Agent]: LLM could not generate a viable flight plan. Aborting.")
            return False

        # 4. Act (Execute the Plan)
        print("\n--- Step 4: ACT (Executing the Plan) ---")
        outcome = self.action.execute_plan(final_plan)
        self.memory.store("flight_outcome", outcome)
        print(f"  [Act]: Outcome recorded: {outcome}")

        # 5. Learn (Reflect and Improve)
        print("\n--- Step 5: LEARN (Reflecting and Improving) ---")
        actual_time = outcome.get("actual_time_hours")
        planned_time = float(final_plan.get("eta_hours", 0))

        if outcome["status"] == "success" and actual_time is not None:
            print("  [Learn]: Flight was successful.")
            past_flights = self.memory.retrieve("past_successful_flights", is_long_term=True)
            past_flights.append({"details": flight_details, "plan": final_plan, "outcome": outcome})
            self.memory.store("past_successful_flights", past_flights, is_long_term=True)

            learning_insight = ""
            if actual_time < planned_time:
                learning_insight = f"Actual time ({actual_time}h) was better than planned ({planned_time}h) due to {outcome.get('event', 'favorable conditions')}. LLM's planning might be slightly conservative or our wind/performance models need refinement. This suggests an opportunity for the LLM to learn optimal 'shortcuts' or account for consistently favorable conditions (Galileo's observation)."
            elif actual_time > planned_time:
                learning_insight = f"Actual time ({actual_time}h) was worse than planned ({planned_time}h) due to {outcome.get('event', 'unexpected factors')}. LLM's planning or our external data sources need to better account for dynamic delays or unforeseen circumstances (Einstein's relativity of time in complex systems). Hinton's principles would guide fine-tuning the LLM to better predict such events."
            else:
                learning_insight = "Actual time matched planned time perfectly. LLM's planning model is highly accurate for these conditions! This validates the current model's predictive power (Newton's deterministic accuracy)."

            print(f"  [Learn - LLM's Self-Reflection Prompt (Conceptual for next LLM phase)]:\n  Prompting LLM with: 'Analyze difference: Planned {planned_time}h, Actual {actual_time}h, Event: {outcome.get('event')}. Provide insights for model improvement.'")
            print(f"    Conceptual LLM insight: {learning_insight}")
            print("    This feedback would then be used to fine-tune the LLM or update its knowledge/prompt engineering to improve future planning accuracy.")
        else:
            print("  [Learn]: Flight was not successful or outcome incomplete. LLM would analyze failure modes and suggest adjustments to its internal logic or data interpretation for next time.")

        print("\n--- Cycle Complete ---")
        return True

# --- Demo Execution ---
if __name__ == "__main__":
    # Changed agent initialization
    flight_agent = FlightPlanningAgent()

    # Scenario 1: YUL to LAX with Boeing 787
    flight_agent.run_flight_planning_cycle(
        goal="Plan a passenger flight from Montreal to Los Angeles for 150 passengers on a Boeing 787",
        flight_details={
            "origin": "YUL",
            "destination": "LAX",
            "aircraft_type": "Boeing 787",
            "pax_count": 150
        }
    )

    print("\n" + "="*80 + "\n")

    # Scenario 2: Montreal to Bangkok - Urgent with Boeing 777
    flight_agent.run_flight_planning_cycle(
        goal="Urgent flight plan from Montreal to Bangkok for 250 executives on a Boeing 777, prioritize speed",
        flight_details={
            "origin": "YUL",
            "destination": "BKK",
            "aircraft_type": "Boeing 777",
            "pax_count": 250,
            "priority": "speed"
        }
    )

    print("\n" + "="*80 + "\n")

    # Scenario 3: JFK to CDG - Cargo with Boeing 767
    flight_agent.run_flight_planning_cycle(
        goal="Plan a cargo flight from JFK to Paris for critical supplies (using Boeing 767)",
        flight_details={
            "origin": "JFK",
            "destination": "CDG",
            "aircraft_type": "Boeing 767",
            "cargo_weight_kg": 5000
        }
    )

Google Generative AI configured successfully using Colab Secrets.
  [LLM Integration]: Initialized LLM client using model: gemini-1.5-flash.

----- Aeronav AI Planner (Powered by gemini-1.5-flash) Agent Initialized -----
  Our flight planning agent, leveraging a **Large Language Model (gemini-1.5-flash)** for dynamic reasoning and complex decision-making.
  Embodying Newton's physics, Galileo's data observation, Einstein's system dynamics,
  and Hinton's deep learning for robust flight planning.
---------------------------------------

[2025-06-29 10:52:14 EDT] Agent's Goal: "Plan a passenger flight from Montreal to Los Angeles for 150 passengers on a Boeing 787"

--- Step 1: INPUT (Gathering Information) ---
  [Input]: Received flight request: {'origin': 'YUL', 'destination': 'LAX', 'aircraft_type': 'Boeing 787', 'pax_count': 150}
  [Input]: Fetched current weather: {'temperature_c': 15, 'wind_kph': 30, 'visibility_km': 8, 'weather': 'partly cloudy with moderate winds'}

--- Step 2 & 